In [1]:
import pandas as pd

In [2]:
data = pd.read_excel("data.xlsx")
distances = pd.read_excel("distances.xlsx")

In [3]:
data.head()

,Unnamed: 0,user_id,request_id,query,ts,last_click_ts,is_reload,is_not_zero_page,has_click,exception_flag,verdict,sample
0,0,41,1324,89636817964,1704201609,1.704202e+09,False,False,True,False,начало,validate
1,1,41,1738,89636817923,1704201718,1.704202e+09,False,False,True,False,продолжение,validate
2,2,41,954,89636817955,1704201823,1.704202e+09,False,False,True,False,продолжение,validate
3,3,41,1998,совкомбанк горячая линия,1704205211,NaN,False,False,False,False,начало,validate
4,4,41,2676,совкомбанк вологда телефон,1704205390,1.704205e+09,False,False,True,False,продолжение,validate


In [5]:
MAX_MINUTES_OF_INACTIVITY = 30
MAX_MINUTES_OF_INACTIVITY_FOR_DISTANT_QUERIES = 4
DISTANT_QUERIES_THRESHOLD = 0.4


class Session(object):
    def __init__(self, request):
        self.user = request.user_id
        self.fisrt_requst_id = request.id
        self.ts_end = None
        self.last_query = None

        self.append(action)

    def append(self, request):
        self.ts_end = request.last_click_ts
        self.last_query = request.query

    def is_in_session(self, request):
        if request.is_not_zero_page:  # не первая страница - всегда в той же сессии
            return True

        if request.ts > self.ts_end + 60 * MAX_MINUTES_OF_INACTIVITY:
            return False
        if request.ts > self.ts_end + 60 * MAX_MINUTES_OF_INACTIVITY_FOR_DISTANT_QUERIES:
            if get_query_dist(self.last_query, request.query) > DISTANT_QUERIES_THRESHOLD:
                return False
        return True

    def get_name(self):
        return self.user_id + "-" + str(self.first_request_id)


def is_technical_request(request):
    if not request.has_click:
        if request.exception_flag or request.is_reload:
            return False
    return True



def add_session_data(requests):
    sessions = list()
    cur_session_id = ""
    for request in requests:
        # Skip technical requests
        if not requests_data[action.get("reqid")].get("in_session"):
            request.session_id = cur_session_id
            continue
        if len(sessions) == 0 or not sessions[-1].is_in_session(request):
            sessions.append(Session(request))
            cur_session_id = sessions[-1].get_name()
        else:
            sessions[-1].append(request)
        request.session_id = cur_session_id

    if len(sessions) > 0:
        for action in actions:
            if request.session_id == "":
                request.session_id = sessions[0].get_name()
            else:
                break
